# Crimes and misdemeanors recorded by the police in France

The following code has been written to retrieve a clearer and easier to use dataset than the one available on data.gouv.fr. The goal is to retrieve only the crimes and offenses having for category:
- Rape on adults
- Rape on minors
- Sexual harassment and other sexual assaults against adults
- Sexual harassment and other sexual assaults against minors
- Sexual assault

In the Excel file provided, these are the index codes from 46 to 50.

The goal here is to be able to express 3 variables:
- Regions (13) or departments (103)
- Type of assault (referenced above, 5 in number)
- Year (from 2011 to 2021, 10)

Dataset : Crimes and offences recorded by the police in France, from 2011 to 2021. Provided by data-gouv.fr.

It can be downloaded at the following [link](https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520).

First, we import the required librairies.

In [2]:
# Basic librairies
import json
from urllib.request import urlopen

# Imported librairies with pip
import pandas as pd
import openpyxl

Then, we make a first cleaning of the dataset. We make some pivots to have the data in a better shape, and we delete the useless columns. Indeed: we just want to keep the columns with index 46 to 50, and the columns with the departments.

In [3]:
filepath = 'crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx'

export_path = 'generated_data/'

def remove_numbers_behind_dot(str):
    return str.split('.')[0]

df = {}

column_rename = json.load(open('column_rename.json'))

for i in range (1, 21):
    df[i] = pd.read_excel(filepath, sheet_name=i, engine='openpyxl')

    # pivot the table
    df[i] = df[i].transpose()

    # set the column headers
    df[i].columns = df[i].iloc[0]
    df[i] = df[i].drop(df[i].index[0])

    # Rename the columns so that "Année 2012 - services de police" becomes "P2012" for instance.
    df[i].columns = df[i].columns.set_names(column_rename[df[i].columns.name])

    # Drop the first 2 columns, only for the police sheet that has 2 columns to drop. Otherwise drop only the first one.
    if df[i].columns.name[0] == 'P':
        df[i] = df[i].drop(df[i].columns[[0, 1]], axis=1)
    else:
        df[i] = df[i].drop(df[i].columns[[0]], axis=1)

    # Remove the numbers behind the dot in the index
    for j in range(2, len(df[i].index)):
        df[i].rename(index={df[i].iloc[j].name: remove_numbers_behind_dot(df[i].iloc[j].name)}, inplace=True)

    # Merge all the lines whose name is the same into one. Also, do a sum of the values.
    df[i] = df[i].groupby(df[i].index).sum()

    # Keep only the columns we need
    df[i] = (df[i].iloc[:, [45, 46, 47, 48, 49]])

    # Remove the line named 'Départements'
    df[i] = df[i].drop('Départements')

merged_df = pd.concat(df.values(), axis=1, keys=[i.columns.name for i in df.values()])

merged_df.columns = merged_df.columns.set_names(['Year', 'Department'])

# Replace NaN values with 0

merged_df = merged_df.fillna(0)

print(merged_df)


Year       P2012                 P2013                  ... G2020           \
Department    46  47  48  49  50    46  47  48  49  50  ...    46   47  48   
01             6  11   9  10  10     8  17  12  29  15  ...    62  122  52   
02            16  29  17  56  36    21  29  19  59  51  ...    47   87  56   
03            10  13  12  13  26     6   8  13  29  20  ...    18   40  24   
04             5   3   2   7  11     4   6   3   9   9  ...    14   18  12   
05             3   1   3   4   8     8   6   5   4  11  ...    13   18   8   
..           ...  ..  ..  ..  ..   ...  ..  ..  ..  ..  ...   ...  ...  ..   
976            0  13   6   8  10     0   6   9   6   0  ...     0    0   0   
987            8  13  16  26  72     8   6  16  20  66  ...    29   46  42   
988           16  10  27  19  28     9  13  17  18  30  ...    41   49  34   
978            0   0   0   0   0     0   0   0   0   0  ...    10    9   9   
986            0   0   0   0   0     0   0   0   0   0  ...     

We create another dataframe that allows us to put the gendarmerie and police years together.

In addition, we add a Total line for each year, which corresponds to the sum of crimes and offenses for each category.

In [4]:
df_total_dict = {}

for year in range(2012, 2022):

    sum_df = merged_df[f'P{year}'] + merged_df[f'G{year}']

    sum_df.columns.name = f'{year}'

    df_total_dict[year] = sum_df

df_total = pd.concat(df_total_dict.values(), axis=1, keys=[i.columns.name for i in df_total_dict.values()])

df_total.columns = df_total.columns.set_names(['Year', 'Department'])

# Add a row with the total of each department for each year
df_total.loc['Total'] = df_total.sum()

print(df_total)

Year        2012                           2013                            \
Department    46    47    48    49     50    46    47    48     49     50   
01            30    70    32    81     68    31    80    47    116     86   
02            44    83    39   130     84    44    81    45    140    120   
03            20    34    25    42     49    11    29    29     66     51   
04            10    11     8    17     32    11    18    14     29     29   
05             7    13     4    26     24    15    12     8     13     20   
...          ...   ...   ...   ...    ...   ...   ...   ...    ...    ...   
987           30    53    50   105    162    42    74    44    104    157   
988           38    24    36    30     87    19    39    31     58     47   
978            9    13     6    14      8     7    10     7     12      4   
986            0     1     0     0      1     0     4     1      1      3   
Total       4749  6145  5612  9797  13400  4941  6282  5891  10630  12647   

We group all the departments together (so we get the regions), and we clean up the dataframe to have the data in a better form.

In [5]:
# Load departements-region.json file
region_dict = {}
for item in json.load(open('departements-region.json')):
    region_dict[item['num_dep']] = item['region_name']

# Copy df_total to df_total_region
df_total_region = df_total.copy()

# Create a new column with the region names based on the dictionary
df_total_region['Région'] = df_total_region.index.map(region_dict)

# Group the dataframe by region name and sum the values
df_total_region = df_total_region.groupby('Région').sum()

# Rename the dataframe
df_total_region.columns = df_total_region.columns.set_names(['Année', 'Crime'])

print(df_total_region)

Année                       2012                         2013             \
Crime                         46   47    48    49    50    46   47    48   
Région                                                                     
Auvergne-Rhône-Alpes         527  705   560  1018  1199   449  704   640   
Bourgogne-Franche-Comté      156  284   222   443   378   170  272   210   
Bretagne                     196  191   253   406   385   206  230   266   
Centre-Val de Loire          142  231   221   428   271   160  266   255   
Corse                         20   24    12    21    30    15   21    10   
Grand Est                    309  469   467   917   859   276  495   470   
Guadeloupe                    37   63    31    51    70    66   67    49   
Guyane                        42   80    21    57    65    45  115    22   
Hauts-de-France              448  718   567  1389  1237   495  739   540   
La Réunion                    69  155    85   157   175    76  138    67   
Martinique  

In [6]:
# Merge all the columns 46, 47, 48, 49, 50 into one with the sum of values.
df_year_only_national = df_total_region.groupby(level=0, axis=1).sum()

df_year_only_national = df_year_only_national.transpose()

Now, we set up a dataframe at the national level, thus grouping all the regions.

In addition, we rename the category with the corresponding text.

In [7]:
# Import the categories.json file and rename
new_categories = json.load(open('categories.json'))

new_categories = {int(k): v for k, v in new_categories.items()}

# Drop every region except Total
df_final = df_total_region.drop(df_total_region.index[:-1])

# Drop the index Région
df_final = df_final.reset_index(drop=True)

# Rename the index 0 to Total
df_final.rename(index={0: 'Total'}, inplace=True)

# Rename the columns
df_final.columns = df_final.columns.set_names(['Year', 'Crime'])

df_national = {}

for year in range(2012, 2022):
    df_national[f'{year}'] = df_final[f'{year}'].transpose()

    # Rename the columns
    df_national[f'{year}'].columns = df_national[f'{year}'].columns.set_names([f'{year}'])

df_national = pd.concat(df_national.values(), axis=1, keys=[i.columns.name for i in df_national.values()])

# Transform the multiindex into a single index, and rename the column of the first index
df_national.columns = df_national.columns.map('_'.join)

# Remove the _Total from the column names
df_national.columns = df_national.columns.str.replace('_Total', '')

df_national = df_national.rename(index=new_categories)

# print the updated index
print(df_national)

                                                    2012  2013  2014  2015  \
Crime                                                                        
Rape on adults                                      1196  1217  1342  1345   
Rape on minors                                       918   829   914  1040   
Sexual harassment and other sexual assaults aga...  1370  1232  1717  1930   
Sexual harassment and other sexual assaults aga...  1194  1332  1554  1512   
Sexual assault                                      3844  2969  2633  2306   

                                                    2016  2017  2018  2019  \
Crime                                                                        
Rape on adults                                      1502  2048  2409  2832   
Rape on minors                                       992  1321  1500  1690   
Sexual harassment and other sexual assaults aga...  2336  2774  3412  3683   
Sexual harassment and other sexual assaults aga...  2023  2209 

Now, with df_region we create a dataframe per region that we can exploit.

In [9]:
# Create a new dataframe for each region
df_region = {}

for region in df_total_region.index:

    df_region[f'{region}'] = df_total_region.loc[f'{region}']

    df_region[f'{region}'] = df_region[f'{region}'].unstack(level='Année')

    df_region[f'{region}'] = df_region[f'{region}'].rename(index=new_categories)

    df_region[f'{region}'] = df_region[f'{region}'].transpose()

    df_region[f'{region}'].index.name = 'Year'

    print(df_region[f'{region}'])

    #df_region[f'{region}'].to_excel(f"{export_path}regions/" + f'{region}.xlsx')

Crime  Rape on adults  Rape on minors  \
Year                                    
2012              527             705   
2013              449             704   
2014              517             779   
2015              601             701   
2016              585             718   
2017              812             935   
2018             1005            1079   
2019             1220            1248   
2020             1375            1412   
2021             1706            1871   

Crime  Sexual harassment and other sexual assaults against adults  \
Year                                                                
2012                                                 560            
2013                                                 640            
2014                                                 574            
2015                                                 850            
2016                                                 991            
2017                    